<a href="https://colab.research.google.com/github/d1p013/testing/blob/master/Assignment_Simple_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTerrier Indexing Demo

This notebook takes you through indexing using [PyTerrier](https://github.com/terrier-org/pyterrier).

## Prerequisites

You will need PyTerrier installed. PyTerrier also needs Java to be installed, and will find most installations.

In [1]:
!pip install python-terrier
#!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

## Init 

You must run `pt.init()` before other pyterrier functions and classes

Optional Arguments:    
 - `version` - terrier IR version e.g. "5.2"    
 - `mem` - megabytes allocated to java e.g. "4096"      
 - `packages` - external java packages for Terrier to load e.g. ["org.terrier:terrier.prf"]
 - `logging` - logging level for Terrier. Defaults to "WARN", use "INFO" or "DEBUG" for more output.

NB: PyTerrier needs Java 11 installed. If it cannot find your Java installation, you can set the `JAVA_HOME` environment variable.

In [2]:
import pyterrier as pt
if not pt.started():
  pt.init()

PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [3]:
 #vaswani_dataset = pt.datasets.get_dataset("vaswani")
 #indexref = vaswani_dataset.get_index()
 #index = pt.IndexFactory.of(indexref)
 #print(index.getCollectionStatistics().toString()) 

In [4]:
#topics = vaswani_dataset.get_topics()
#topics.head(5) 

In [5]:
#retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"}, num_results=10)
#retr.setControl("wmodel", "TF_IDF")
#retr.setControls({"wmodel": "TF_IDF"})
#res=retr.transform(topics)
#res 

 ## Importing dataset from Google Drive

Using built-in function of Google Colab, we can easily import the dataset which has been uploaded onto Google Drive beforehand

need a Google account to download the data

In [6]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID which is residing on Google Drive
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# file_id = '1YkltHAl3Ro9xv25EkGB56h271_STNlPA' #'AI6122_Dataset_B1.csv'
file_id = '1aVXMJ_luTXISxMwP5_Bt2xE0ObXqEsQ2' #"Dataset_B1to8"

downloaded = drive.CreateFile({'id': file_id})
#print('Downloaded content "{}"'.format(downloaded.GetContentString()))
#downloaded.GetContentFile('AI6122_Dataset_B1.csv')
downloaded.GetContentFile('Dataset_B1to8.csv')

## Loading dataset (csv) into Pandas dataframe

pyTerrier makes it easy to index standard Python data structures, particularly [Pandas dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

importing dataset, which is scale down version of the original `review.json` file as the original file is too large to load into Google Colab memory


In [7]:
## load data into df
import pandas as pd
#df = pd.read_csv("AI6122_Dataset_B1.csv", dtype = str)
df = pd.read_csv("Dataset_B1to8.csv", dtype = str)

try :
  del docno 
except:
  pass

##docno = list(range(1,len(df)+1))
##df['docno'] = docno
##df = df.astype({"docno" : str})
#print(type(df["docno"][0]))

docno =[]
for idx in range(1,len(df)+1):
  docno.append("d" + str(idx))

df["docno"] = docno


## Time taken for indexing

Tracking the time to complete indexing for each 10% incremental of documents. The index folder is cleared before each indexing and time and is taken before and after indexing, the difference being the time taken to index


In [ ]:
import time

Tcollect = []
for idx in range(1,11):
  #print(idx)
  df_resize = df.iloc[:int(len(df)/(10/idx)),:]
  !rm -rf ./pd_index
  pd_indexer = pt.DFIndexer("./pd_index", overwrite=True, verbose=True)
  Tstart = time.perf_counter()
  indexref = pd_indexer.index(df_resize["text"], df_resize)
  Tend = time.perf_counter()
  print(str(10*idx)+f"% df search completed in {Tend - Tstart:0.4f} seconds")
  Tcollect.append(Tend - Tstart)



  0%|          | 0/692 [00:00<?, ?documents/s]

10% df search completed in 3.3944 seconds


  0%|          | 0/1385 [00:00<?, ?documents/s]

20% df search completed in 3.5503 seconds


  0%|          | 0/2078 [00:00<?, ?documents/s]

...

12:17:38.490 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.522 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.575 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.589 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.675 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_pro

12:17:38.725 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.814 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.884 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_proxy.pxi", line 50, in jnius.PythonJavaClass.invoke
  File "jnius/jnius_proxy.pxi", line 76, in jnius.PythonJavaClass._invoke
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 534, in next
    meta = self.meta.__next__()
StopIteration
Traceback (most recent call last):
  File "jnius/jnius_pro

12:17:38.912 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:38.990 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:39.026 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document
12:17:39.126 [main] WARN org.terrier.structures.indexing.Indexer - skipping null document


In [ ]:
import matplotlib.pyplot as plt

plotdf = pd.DataFrame({
    'Percentage Completed':['10%', '20%', '30%', '40%', '50%', 
                            '60%', '70%', '80%', '90%', '100%'],
    'Time(s)':Tcollect
    })

# a scatter plot comparing num_children and num_pets
plotdf.plot(kind='line',x='Percentage Completed',y='Time(s)',color='blue')
plt.show()


## Indexing a Pandas dataframe

We can use a `pt.DFIndexer()` object to do indexing for Pandas dataframe

In [ ]:
#import pandas as pd
!rm -rf ./pd_index
pd_indexer = pt.DFIndexer("./pd_index", overwrite=True, verbose=True)
# no stemming or stopwords
pd_indexer.setProperty("termpipelines", "")

# optionally modify properties
# index_properies = {"block.indexing":"true", "invertedfile.lexiconscanner":"pointers"}
# indexer.setProperties(**index_properies)
# no stemming or stopwords
# indexer.setProperty("termpipelines", "")

Then there are a number of options to index the dataframe:    
The first argument should always a pandas.Series object of Strings, which specifies the body of each document.    
Any arguments after that are for specifying metadata.

We can view more useful information from the indexed objects using a indexref.getCollectionsStatistics()

In [ ]:
import time
# no metadata
# pd_indexer.index(df["text"])

# Add metadata fields as Pandas.Series objects, with the name of the Series object becoming the name of the meta field.
#indexref = pd_indexer.index(df["text"], df["docno"], df["review_id"], df["user_id"], df["business_id"], df["stars"], df["useful"], df["funny"], df["cool"])
Tstart = time.perf_counter()
indexref = pd_indexer.index(df["text"], df)
Tend = time.perf_counter()
print(f"search completed in {Tend - Tstart:0.4f} seconds")
indexinfo = pt.IndexFactory.of(indexref)
print(indexinfo.getCollectionStatistics().toString()) 
# pd_indexer.index(df["text"], df["docno"], df["url"])

# Add metadata fields as lists to a keyword arguement
# pd_indexer.index(df["text"], docno=["1","2","3"], url=["url1", "url2", "url3"])

# Add the metadata fields with a dictionary
# meta_fields={"docno":["1","2","3"],"url":["url1", "url2", "url3"]}
# pd_indexer.index(df["text"], **meta_fields)

# Add the entire dataframe as metadata
# pd_indexer.index(df["text"], df)

In the above example, the indexed collection had 6928 documents, which contained 388543-word occurrences. Out of which 12049 were identified as unique words. The total postings in the inverted index are 323743. 

The whole datafame is being index, with the `"text"` field being searchable while the remaining (e.g. `"review_id"`, `"user_id"`, `"business_id"`, `"stars"`, `"date"`, `"useful"`, `"funny"`, `"cool"`, `"text"`) as metadata which can be displayed when called upon.

pyTerrier perform standard stopwords removal and applies Porter's stemmer by default, and it is applicable in this notebook as well.
EnglishTokeniser is the default tokeniser and case-folding to lower case is applied during Tokenization

## Retrieval

Retrieval takes place using the `BatchRetrieve` object, by invoking `transform()` method for one or more queries. For a quick test, you can give just pass your query to `transform()`. 

BatchRetrieve will return the results as a Pandas dataframe.


In [ ]:
#pt.BatchRetrieve(indexref).search("so many")

In [ ]:

#this ranker will make the candidate set of documents for each query
#BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"}, num_results=5)
#these rankers we will use to re-rank the BM25 results
#TF_IDF = pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"}, num_results=5)
#PL2 =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"}, num_results=5)

#pipe = BM25 >> (TF_IDF ** PL2)
#pipe.transform("really cute restaurant") 


In [ ]:
#pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"}, num_results=5).search("Really cute restaurant")


In [ ]:
#pt.BatchRetrieve(indexref, wmodel="BM25", properties={"termpipelines" : "Stopwords,PorterStemmer"})
#pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"}, properties={"termpipelines" : ""}, num_results=10).search("Really cute restaurant")
#pt.BatchRetrieve(indexref, metadata=["business_id", "stars"], num_results=10).search("Really cute restaurant")


However, most IR experiments, will use a set of queries. You can pass such a set using a data frame for input.

In [ ]:
#import pandas as pd
#topics = pd.DataFrame([["q1", "Really cute restaurant"], ["q2", "restaurant"]],columns=['qid','query'])
#pt.BatchRetrieve(indexref, metadata=["text"], num_results=5).transform(topics)

## Simple Search Engine

Prompting user input for search string
Prompting user input for the Top N results to return
Display the time taken to complete the search
Display the search results

In [ ]:
import time
import pandas as pd
pd.reset_option('^display.', silent=True)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

search_str = input("Please enter your search string: ")
#print("Search string: ", search_str)
TopN = input("Please enter number results to display: ")
#print("Top N results: ", TopN)
Tstart = time.perf_counter()
topics = pd.DataFrame([["q1", search_str]],columns=['qid','query'])
results = pt.BatchRetrieve(indexref, metadata=["text"], num_results=int(TopN)).transform(topics)
Tend = time.perf_counter()
print(f"search completed in {Tend - Tstart:0.4f} seconds")

if len(results) == 0:
  print("no result found!")
#else:
  #print("len: ", len(results))
  #print(results)

results

Thats the end of the indexing tutorial - you can continue with other example tutorials.